Todo

In [9]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import dash_bootstrap_components as dbc

# === 1. Cargar y preparar el DataFrame
df = pd.read_excel("data/empleabilidad.xlsx", sheet_name="Titulos")
df.columns = df.columns.str.upper().str.strip()
df["FECHA DE REGISTRO"] = pd.to_datetime(
    df["FECHA DE REGISTRO"], dayfirst=True, errors="coerce"
)
df["NIVEL ACADÉMICA"] = df["NIVEL ACADÉMICA"].str.upper().str.strip()
df["TIPO_TITULO"] = df["NIVEL ACADÉMICA"].apply(
    lambda x: "Pregrado" if "TERCER" in x else "Posgrado"
)

# === 2. Generar continuidad
df_sorted = df.sort_values(["IDENTIFICACION", "FECHA DE REGISTRO"])
df_sorted["ORDEN"] = df_sorted.groupby("IDENTIFICACION").cumcount() + 1

# Extraer pregrados
pregrados = (
    df_sorted[df_sorted["TIPO_TITULO"] == "Pregrado"]
    .groupby("IDENTIFICACION")
    .first()
    .reset_index()
)
pregrados = pregrados.rename(
    columns={
        "TÍTULO HOMOLOGADO": "PREGRADO",
        "FECHA DE REGISTRO": "FECHA_PREGRADO",
        "CARRERA": "PRE_CARRERA",
        "FACULTAD": "PRE_FACULTAD",
    }
)

# Extraer posgrados
posgrados = df_sorted[df_sorted["TIPO_TITULO"] == "Posgrado"].copy()
posgrados["ORDEN"] = posgrados.groupby("IDENTIFICACION").cumcount() + 1

# Combinar
df_continuidad = posgrados.merge(
    pregrados[
        ["IDENTIFICACION", "PREGRADO", "FECHA_PREGRADO", "PRE_CARRERA", "PRE_FACULTAD"]
    ],
    on="IDENTIFICACION",
    how="left",
)
df_continuidad["TIEMPO_DESDE_PREGRADO"] = (
    df_continuidad["FECHA DE REGISTRO"] - df_continuidad["FECHA_PREGRADO"]
).dt.days / 365.25
df_continuidad = df_continuidad.rename(
    columns={
        "INSTITUCIÓN DE EDUCACIÓN SUPERIOR": "POS_UNIVERSIDAD",
        "TÍTULO HOMOLOGADO": "POSGRADO",
    }
)

# === 3. Inicializar app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# === 4. Layout
app.layout = dbc.Container(
    [
        html.H2("🎓 Análisis de Continuidad Educativa", className="my-4"),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Filtrar por Facultad de Pregrado:"),
                        dcc.Dropdown(
                            id="filtro_facultad",
                            options=[
                                {"label": f, "value": f}
                                for f in sorted(
                                    df_continuidad["PRE_FACULTAD"].dropna().unique()
                                )
                                if f.strip().upper() != "SIN REGISTRO"
                                and f.strip() != ""
                            ],
                            placeholder="Selecciona una facultad",
                        ),
                    ],
                    width=6,
                ),
                dbc.Col(
                    [
                        html.Label("Filtrar por Carrera de Pregrado:"),
                        dcc.Dropdown(
                            id="filtro_carrera", placeholder="Selecciona una carrera"
                        ),
                    ],
                    width=6,
                ),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tasa de Continuidad"),
                            dbc.CardBody(
                                html.H4(id="tasa_continuidad", className="card-title")
                            ),
                        ]
                    )
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tasa de Recompra UDLA"),
                            dbc.CardBody(
                                html.H4(id="tasa_recompra", className="card-title")
                            ),
                        ]
                    )
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tiempo al 1er Posgrado"),
                            dbc.CardBody(
                                html.H4(id="tiempo_1er", className="card-title")
                            ),
                        ]
                    )
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tiempo al 2do Posgrado"),
                            dbc.CardBody(
                                html.H4(id="tiempo_2do", className="card-title")
                            ),
                        ]
                    )
                ),
            ]
        ),
    ],
    fluid=True,
)


# === 5. Callback para actualizar dropdown de carreras
@app.callback(Output("filtro_carrera", "options"), Input("filtro_facultad", "value"))
def actualizar_dropdown_carreras(facultad):
    if facultad:
        carreras = (
            df_continuidad[df_continuidad["PRE_FACULTAD"] == facultad]["PRE_CARRERA"]
            .dropna()
            .unique()
        )

    else:
        carreras = df_continuidad["PRE_CARRERA"].dropna().unique()
    return [{"label": c, "value": c} for c in sorted(carreras)]


# === 6. Callback para actualizar métricas
@app.callback(
    [
        Output("tasa_continuidad", "children"),
        Output("tasa_recompra", "children"),
        Output("tiempo_1er", "children"),
        Output("tiempo_2do", "children"),
    ],
    [Input("filtro_facultad", "value"), Input("filtro_carrera", "value")],
)
def actualizar_metrica(facultad, carrera):
    data = df_continuidad.copy()

    if facultad:
        data = data[data["PRE_FACULTAD"] == facultad]
    if carrera:
        data = data[data["PRE_CARRERA"] == carrera]

    if facultad or carrera:
        base = df.copy()
        if facultad:
            base = base[base["FACULTAD"] == facultad]
        if carrera:
            base = base[base["CARRERA"] == carrera]
        base = base[base["TIPO_TITULO"] == "Pregrado"]
    else:
        base = df[df["TIPO_TITULO"] == "Pregrado"]

    total_base = base["IDENTIFICACION"].nunique()
    total_continua = data["IDENTIFICACION"].nunique()
    print(total_continua)
    print(total_base)
    tasa_cont = round(100 * total_continua / total_base, 1) if total_base else 0

    total_recompra = data[data["POS_UNIVERSIDAD"] == "UNIVERSIDAD DE LAS AMERICAS"][
        "IDENTIFICACION"
    ].nunique()
    tasa_recompra = (
        round(100 * total_recompra / total_continua, 1) if total_continua else 0
    )

    tiempo_1 = round(data[data["ORDEN"] == 1]["TIEMPO_DESDE_PREGRADO"].mean(), 1)
    tiempo_2 = round(data[data["ORDEN"] == 2]["TIEMPO_DESDE_PREGRADO"].mean(), 1)

    return f"{tasa_cont}%", f"{tasa_recompra}%", f"{tiempo_1} años", f"{tiempo_2} años"


# === 7. Ejecutar
if __name__ == "__main__":
    app.run(debug=True, port=8052)

13086
35899
13086
35899


Filtros

In [2]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc

# === 1. Cargar y preparar los datos
df = pd.read_excel("data/empleabilidad.xlsx", sheet_name="Titulos")
df.columns = df.columns.str.upper().str.strip()
df["FECHA DE REGISTRO"] = pd.to_datetime(
    df["FECHA DE REGISTRO"], dayfirst=True, errors="coerce"
)
df["NIVEL ACADÉMICA"] = df["NIVEL ACADÉMICA"].str.upper().str.strip()
df["TIPO_TITULO"] = df["NIVEL ACADÉMICA"].apply(
    lambda x: "Pregrado" if "TERCER" in x else "Posgrado"
)

# Extraer datos de pregrado y posgrado
pregrados = df[df["TIPO_TITULO"] == "Pregrado"]
posgrados = df[df["TIPO_TITULO"] == "Posgrado"]

# === 2. Inicializar la app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# === 3. Layout
app.layout = dbc.Container(
    [
        html.H2(
            "📘 Visualización de Estudiantes - Filtros Dinámicos", className="my-4"
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Facultad:"),
                        dcc.Dropdown(
                            id="filtro_facultad",
                            options=[
                                {"label": f, "value": f}
                                for f in sorted(pregrados["FACULTAD"].dropna().unique())
                                if f.strip().upper() != "SIN REGISTRO"
                                and f.strip() != ""
                            ],
                            placeholder="Selecciona una facultad",
                        ),
                    ],
                    width=6,
                ),
                dbc.Col(
                    [
                        html.Label("Carrera:"),
                        dcc.Dropdown(
                            id="filtro_carrera", placeholder="Selecciona una carrera"
                        ),
                    ],
                    width=6,
                ),
            ],
            className="mb-4",
        ),
        html.H5("👨‍🎓 Estudiantes de Pregrado:"),
        dash_table.DataTable(
            id="tabla_pregrado", page_size=10, style_table={"overflowX": "auto"}
        ),
        html.Hr(),
        html.H5("🎓 Estudiantes que continuaron a Posgrado:"),
        dash_table.DataTable(
            id="tabla_posgrado", page_size=10, style_table={"overflowX": "auto"}
        ),
    ],
    fluid=True,
)


# === 4. Callback para actualizar carreras según facultad
@app.callback(Output("filtro_carrera", "options"), Input("filtro_facultad", "value"))
def actualizar_carreras(facultad):
    if facultad:
        carreras = (
            pregrados[pregrados["FACULTAD"] == facultad]["CARRERA"].dropna().unique()
        )
    else:
        carreras = pregrados["CARRERA"].dropna().unique()
    return [{"label": c, "value": c} for c in sorted(carreras)]


# === 5. Callback para actualizar tablas
@app.callback(
    [
        Output("tabla_pregrado", "data"),
        Output("tabla_pregrado", "columns"),
        Output("tabla_posgrado", "data"),
        Output("tabla_posgrado", "columns"),
    ],
    [Input("filtro_facultad", "value"), Input("filtro_carrera", "value")],
)
def actualizar_tablas(facultad, carrera):
    # Filtro pregrado
    filtro = pregrados.copy()
    if facultad:
        filtro = filtro[filtro["FACULTAD"] == facultad]
    if carrera:
        filtro = filtro[filtro["CARRERA"] == carrera]

    columnas_pre = [{"name": col, "id": col} for col in filtro.columns]
    datos_pre = filtro.to_dict("records")

    # IDs únicos para buscar sus posgrados
    ids = filtro["IDENTIFICACION"].unique()
    pos = posgrados[posgrados["IDENTIFICACION"].isin(ids)]
    columnas_pos = [{"name": col, "id": col} for col in pos.columns]
    datos_pos = pos.to_dict("records")

    return datos_pre, columnas_pre, datos_pos, columnas_pos


# === 6. Ejecutar
if __name__ == "__main__":
    app.run(debug=True, port=8053)

[2025-07-02 14:11:49,097] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\andreidavid.flores\Downloads\Work\Graficos\.venv\Lib\site-packages\pandas\core\indexes\base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'CARRERA'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\andreidavid.flores\Downloads\Work\Graficos\.venv\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request

asdf

In [ ]:
import pandas as pd
import dash
from dash import html, dcc, Input, Output
import dash_bootstrap_components as dbc

# === 1. Cargar y preparar el DataFrame
df = pd.read_excel("data/empleabilidad.xlsx", sheet_name="Titulos")
df.columns = df.columns.str.upper().str.strip()

## DATA

# Normalizar columna de nivel académico
df["NIVEL ACADÉMICA"] = df["NIVEL ACADÉMICA"].str.upper().str.strip()

# Clasificar tipo de título
df["TIPO_TITULO"] = df["NIVEL ACADÉMICA"].apply(
    lambda x: "Pregrado" if x.startswith("TERCER") else ("Posgrado" if x.startswith("CUARTO") else "Otro")
)

# === 2. Inicializar la app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# === 3. Layout con filtros
app.layout = dbc.Container(
    [
        html.H2("🎓 Explorador de Datos de Empleabilidad", className="my-4"),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Filtrar por Facultad:"),
                        dcc.Dropdown(
                            id="filtro_facultad",
                            options=[
                                {"label": f, "value": f}
                                for f in sorted(df["FACULTAD"].dropna().unique())
                                if f.strip().upper() != "SIN REGISTRO"
                            ],
                            placeholder="Selecciona una facultad",
                        ),
                    ],
                    width=6,
                ),
                dbc.Col(
                    [
                        html.Label("Filtrar por Carrera:"),
                        dcc.Dropdown(
                            id="filtro_carrera",
                            placeholder="Selecciona una carrera",
                        ),
                    ],
                    width=6,
                ),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tasa de Continuidad"),
                            dbc.CardBody(
                                html.H4(id="tasa_continuidad", className="card-title")
                            ),
                        ]
                    ),
                    width=3,
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tasa de Recompra UDLA"),
                            dbc.CardBody(
                                html.H4(id="tasa_recompra", className="card-title")
                            ),
                        ]
                    ),
                    width=3,
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tiempo al 1er Posgrado"),
                            dbc.CardBody(
                                html.H4(id="tiempo_1er", className="card-title")
                            ),
                        ]
                    ),
                    width=3,
                ),
                dbc.Col(
                    dbc.Card(
                        [
                            dbc.CardHeader("Tiempo al 2do Posgrado"),
                            dbc.CardBody(
                                html.H4(id="tiempo_2do", className="card-title")
                            ),
                        ]
                    ),
                    width=3,
                ),
            ],
            className="mb-4",
        ),
        html.Div(id="debug-output"),  # Esto es útil para pruebas
    ],
    fluid=True,
)


# === 4. Callback para actualizar carreras según la facultad
@app.callback(Output("filtro_carrera", "options"), Input("filtro_facultad", "value"))
def actualizar_dropdown_carreras(facultad):
    if facultad:
        carreras = df[df["FACULTAD"] == facultad]["CARRERA"].dropna().unique()
    else:
        carreras = df["CARRERA"].dropna().unique()
    return [{"label": c, "value": c} for c in sorted(carreras)]


@app.callback(
    [
        Output("tasa_continuidad", "children"),
        Output("tasa_recompra", "children"),
        Output("tiempo_1er", "children"),
        Output("tiempo_2do", "children"),
    ],
    [Input("filtro_facultad", "value"), Input("filtro_carrera", "value")],
)

def actualizar_metricas(facultad, carrera):
    # === 1. Filtrar pregrados UDLA
    pregrado_udla = df[
        (df["TIPO_TITULO"] == "Pregrado")
        & (df["INSTITUCIÓN DE EDUCACIÓN SUPERIOR"] == "UNIVERSIDAD DE LAS AMERICAS")
    ]
    if facultad:
        pregrado_udla = pregrado_udla[pregrado_udla["FACULTAD"] == facultad]
    if carrera:
        pregrado_udla = pregrado_udla[pregrado_udla["CARRERA"] == carrera]

    # === 2. Posgrados (sin filtrar por facultad/carrera)
    posgrados = df[df["TIPO_TITULO"] == "Posgrado"]

    # === 3. Cruces
    ids_pregrado_udla = set(pregrado_udla["IDENTIFICACION"])
    ids_posgrado = set(posgrados["IDENTIFICACION"])
    ids_continua = ids_pregrado_udla & ids_posgrado  # Tasa de continuidad

    # === 4. Tasa de continuidad
    total_pregrado_udla = len(ids_pregrado_udla)
    total_continua = len(ids_continua)
    tasa_cont = (
        round(100 * total_continua / total_pregrado_udla, 1)
        if total_pregrado_udla
        else 0
    )

    # === 5. Tasa de recompra (pregrado y posgrado ambos en UDLA)
    ids_posgrado_udla = set(
        posgrados[
            posgrados["INSTITUCIÓN DE EDUCACIÓN SUPERIOR"]
            == "UNIVERSIDAD DE LAS AMERICAS"
        ]["IDENTIFICACION"]
    )
    ids_recompra = ids_pregrado_udla & ids_posgrado_udla
    tasa_recompra = (
        round(100 * len(ids_recompra) / total_continua, 1) if total_continua else 0
    )

    # === Tiempo al 1.er posgrado ===============================================

    # 1. Primer posgrado por graduado
    primer_posgrados = (
        posgrados[posgrados["IDENTIFICACION"].isin(ids_continua)]
        .sort_values(["IDENTIFICACION", "FECHA DE REGISTRO"])
        .groupby("IDENTIFICACION")
        .first()                       # primer título posgrado
        .reset_index()                 # ← recuperamos IDENTIFICACION como columna
        .rename(columns={"FECHA DE REGISTRO": "FECHA_POSGRADO"})
        [["IDENTIFICACION", "FECHA_POSGRADO"]]
    )

    # 2. Pregrado UDLA (primero que aparece)
    pregrados_base = (
        pregrado_udla[pregrado_udla["IDENTIFICACION"].isin(ids_continua)]
        .sort_values(["IDENTIFICACION", "FECHA DE REGISTRO"])
        .groupby("IDENTIFICACION")
        .first()
        .reset_index()
        .rename(columns={"FECHA DE REGISTRO": "FECHA_PREGRADO"})
        [["IDENTIFICACION", "FECHA_PREGRADO"]]
    )

    # 3. Unión y diferencia
    df_tiempos1 = pd.merge(primer_posgrados, pregrados_base, on="IDENTIFICACION")
    df_tiempos1["FECHA_PREGRADO"]  = pd.to_datetime(df_tiempos1["FECHA_PREGRADO"])
    df_tiempos1["FECHA_POSGRADO"]  = pd.to_datetime(df_tiempos1["FECHA_POSGRADO"])

    df_tiempos1["TIEMPO_ANIOS"] = (
        (df_tiempos1["FECHA_POSGRADO"] - df_tiempos1["FECHA_PREGRADO"])
        .dt.total_seconds() / (365.25 * 24 * 3600)
    )
    tiempo_1 = round(df_tiempos1["TIEMPO_ANIOS"].mean(), 1) if not df_tiempos1.empty else "—"

    # === Tiempo al 2.º posgrado ===============================================

    # 1. Segundo posgrado (nth(1) = índice 1)  
    segundo_posgrados = (
        posgrados[posgrados["IDENTIFICACION"].isin(ids_continua)]
        .sort_values(["IDENTIFICACION", "FECHA DE REGISTRO"])
        .groupby("IDENTIFICACION")
        .nth(1)                        # segundo título
        .reset_index()
        .rename(columns={"FECHA DE REGISTRO": "FECHA_2DO_POSGRADO"})
        [["IDENTIFICACION", "FECHA_2DO_POSGRADO"]]
    )

    # Unión: 1.º ↔ 2.º posgrado
    df_tiempos2 = pd.merge(
        segundo_posgrados,
        primer_posgrados[["IDENTIFICACION", "FECHA_POSGRADO"]],
        on="IDENTIFICACION",
        how="inner"
    )

    # Asegurar datetime
    df_tiempos2["FECHA_POSGRADO"]      = pd.to_datetime(df_tiempos2["FECHA_POSGRADO"])
    df_tiempos2["FECHA_2DO_POSGRADO"]  = pd.to_datetime(df_tiempos2["FECHA_2DO_POSGRADO"])

    # Diferencia en años
    df_tiempos2["TIEMPO_ANIOS"] = (
        (df_tiempos2["FECHA_2DO_POSGRADO"] - df_tiempos2["FECHA_POSGRADO"])
        .dt.total_seconds() / (365.25 * 24 * 3600)
    )

    # Promedio
    tiempo_2 = round(df_tiempos2["TIEMPO_ANIOS"].mean(), 1) if not df_tiempos2.empty else "—"

    # (los prints de depuración los puedes dejar si te sirven)
    print("⏱️ 1.er posgrado – sample:\n", df_tiempos1.head(10))
    print("⏱️ 2.º posgrado – sample:\n", df_tiempos2.head(10))

    return f"{tasa_cont}%", f"{tasa_recompra}%", f"{tiempo_1} años", f"{tiempo_2} años"


# === 5. Ejecutar
if __name__ == "__main__":
    app.run(debug=True, port=8052)

⏱️ 1.er posgrado – sample:
    IDENTIFICACION FECHA_POSGRADO FECHA_PREGRADO  TIEMPO_ANIOS
0       102124609     2008-06-16     2002-09-20      5.738535
1       102423514     2004-05-05     2002-11-13      1.475702
2       103127411     2022-07-27     2020-10-28      1.744011
3       103565008     2016-04-29     2012-11-07      3.474333
4       103584868     2022-07-14     2014-04-08      8.265572
5       103651295     2019-06-07     2016-08-31      2.765229
6       103759700     2023-05-23     2016-09-01      6.721424
7       103929295     2020-04-20     2016-11-15      3.427789
8       103970091     2020-02-10     2016-04-15      3.822040
9       104179841     2018-04-11     2016-09-21      1.552361
⏱️ 2.º posgrado – sample:
    IDENTIFICACION FECHA_2DO_POSGRADO FECHA_POSGRADO  TIEMPO_ANIOS
0       103651295         2021-12-07     2019-06-07      2.502396
1       104179841         2021-12-15     2018-04-11      3.679671
2       104459748         2017-09-20     2013-04-10      4.446270

In [74]:
from datetime import datetime

fecha_pregrado = datetime.strptime("06/04/2018", "%d/%m/%Y")
fecha_posgrado = datetime.strptime("18/07/2022", "%d/%m/%Y")

diferencia = fecha_posgrado - fecha_pregrado
anios = diferencia.days / 365.25
print(round(anios, 2))  # → 1.61 años

4.28
